Plot the SST in the Canary Current Upwelling System.

In [1]:
import os
import glob
import filament
import datetime
import netCDF4
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from importlib import reload

In [2]:
logger = logging.getLogger("filamentPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 1

In [15]:
datadir = "/data/SST/CapeGhir/2plot/"
figdir = "/data/SST/CapeGhir/figures/"
swotfile = "/data/SWOT/swot292_science.txt"
visibledir = "/data/Visible/"
datafilelist = sorted(glob.glob(os.path.join(datadir, "*SST*.nc")))
nfiles = len(datafilelist)
logger.info("Processing {} files".format(nfiles))

INFO:filamentPlot:Processing 2 files


In [16]:
if not(os.path.exists(figdir)):
    os.makedirs(figdir)

In [17]:
coordinates = (-19., -6., 25., 35.)
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            epsg=4326, resolution='h')

In [28]:
for datafile in datafilelist:
    print(datafile)
    with netCDF4.Dataset(datafile) as nc:
        proclevel = nc.processing_level
    sst = filament.SST()
    
    # Select the reader according to processing level
    if proclevel == "L3 Mapped":
        logger.info("Level-3 data")
        sst.read_from_oceancolorL3(datafile, coordinates)
    elif proclevel == "L2":
        logger.info("Level-2 data")
        sst.read_from_oceancolorL2(datafile)
        sst.field = np.ma.masked_where(sst.qflag > 2, sst.field)
    
    # Plot without visible
    figname = os.path.join(figdir, sst.get_figname())
    sst.make_plot(m, figname=figname, vmin=16., vmax=21, shrink=0.7)
    
    # Plot quality flag 
    figname = os.path.join(figdir, sst.get_figname() + "_qf")
    sst.make_plot_qf(m, figname=figname, shrink=0.7)
    
    
    # With visible image background
    visiblefile = os.path.join(visibledir, "VIIRS{}.tiff".format(sst.date.strftime("-%Y-%m-%dT00_00_00Z")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible")))
    sst.make_plot2(m, figname=figname, visibleim=visibleim, vmin=16., vmax=21.)

INFO:filamentPlot:Level-2 data


/data/SST/CapeGhir/2plot/SNPP_VIIRS.20191227T021801.L2.SST.NRT.nc


INFO:filamentPlot:Level-2 data


/data/SST/CapeGhir/2plot/SNPP_VIIRS.20191227T133600.L2.SST.NRT.nc


In [27]:
visiblefile

'/data/Visible/VIIRS-2019-12-27T00_00_00Z.tiff'

## GHRSST

In [7]:
datadir2 = "/data/SST/GHRSST/"
datafilelist2 = sorted(glob.glob(os.path.join(datadir2, "20170904*.nc")))
nfiles = len(datafilelist2)
logger.info("Processing {} files".format(nfiles))

INFO:filamentPlot:Processing 1 files


In [8]:
for datafile in datafilelist2:
    logger.info(datafile)
    sst = filament.SST()
    sst.read_from_ghrsst(datafile)
    sst.field = np.ma.masked_where(sst.qflag != 5, sst.field)
    
    # Plot without visible
    figname = os.path.join(figdir, sst.get_figname())
    sst.make_plot2(m, figname=figname)
    
    # With visible image background
    visiblefile = os.path.join(visibledir, "VIIRS_{}.tiff".format(sst.date.strftime("%Y%m%d")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible")))
    sst.make_plot2(m, figname=figname, visibleim=visibleim)
    
    # With visible image background and SWOT
    swot = filament.Swot()
    swot.read_from(swotfile)
    swot.select_domain(coordinates)
    visiblefile = os.path.join(visibledir, "VIIRS_{}.tiff".format(sst.date.strftime("%Y%m%d")))
    if os.path.exists(visiblefile):
        visibleim = filament.Visible()
        visibleim.read_from(visiblefile)
        visibleim.extract_area(coordinates)
    figname = os.path.join(figdir, "_".join((sst.get_figname(), "visible", "swot")))
    sst.make_plot2(m, figname=figname, visibleim=visibleim, swot=swot)

INFO:filamentPlot:/data/SST/GHRSST/20170904140000-OSPO-L2P_GHRSST-SSTskin-VIIRS_NPP-ACSPO_V2.40-v02.0-fv01.0.nc


TypeError: '>=' not supported between instances of 'NoneType' and 'float'